In [ ]:
#!pip install PyPDF2
#!pip install tinydb
#!pip install langchain
#!pip install langchain-community
#!pip install pandas


  Using cached langchain_core-0.3.47-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_text_splitters-0.3.7-py3-none-any.whl.metadata (1.9 kB)
  Using cached langsmith-0.3.18-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached sqlalchemy-2.0.39-cp313-cp313-macosx_11_0_arm64.whl.metadata (9.6 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached PyYAML-6.0.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached orjson-3.10.15-cp313-cp313-macosx_10_15_x86_64.macosx_11_0_arm64.macosx_10_15_universal2.whl.metadata (41 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cac

In [3]:
# %% [code]
import os
#from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import PyPDF2
from tinydb import TinyDB, Query
from langchain.chat_models import AzureChatOpenAI

# Configuración del LLM (ajusta el parámetro 'temperature' o agrega tus credenciales si es necesario)


llm = AzureChatOpenAI(
    openai_api_base="https://sg-openai.openai.azure.com/",
    openai_api_key="647e7eaa5678417f8667203df58216ee",  # Reemplaza con tu clave de API
    openai_api_version="2023-03-15-preview",  # Ajusta según la versión que estés usando
    deployment_name="gpt-4o",
    temperature=0.2
)

# Inicialización de TinyDB (la base se almacenará en el archivo history.json)
db = TinyDB("ResCTM.json")
records_table = db.table("records")


/Users/joaquintschopp/ProyectosPY/AgentesAI/.venv/lib/python3.13/site-packages/langchain_community/chat_models/azure_openai.py:174: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://sg-openai.openai.azure.com/ to https://sg-openai.openai.azure.com/openai.
  warnings.warn(
/Users/joaquintschopp/ProyectosPY/AgentesAI/.venv/lib/python3.13/site-packages/langchain_community/chat_models/azure_openai.py:181: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
/Users/joaquintschopp/ProyectosPY/AgentesAI/.venv/lib/python3.13/site-packages/langchain_community/chat_models/azure_openai.py:189: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base

# Funciones de procesamiento

In [4]:
# %% [code]
def pdf_to_text(pdf_path):
    """
    Extrae el texto de un archivo PDF utilizando PyPDF2.
    """
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text


# 1. Corrección de la transcripción usando LangChain

In [5]:
# %% [code]
correction_prompt = PromptTemplate(
    input_variables=["text"],
    template="Corrige la transcripción del siguiente texto:\n\n{text}\n\nTexto corregido:"
)

def correct_transcription(text):
    """
    Utiliza un LLM para corregir la transcripción extraída del PDF.
    """
    chain = LLMChain(llm=llm, prompt=correction_prompt)
    corrected = chain.run(text=text)
    return corrected


# 2. Extracción de datos y generación del resumen

In [6]:
# Prompt adaptado con instrucciones específicas
extraction_prompt = PromptTemplate(
    input_variables=["text", "template_info"],
    template=(
        "Utilizando el siguiente template:\n"
        "{template_info}\n\n"
        "Extrae únicamente los datos solicitados y genera un resumen ejecutivo breve (máximo 30 palabras) "
        "del tema principal del texto, sin incluir detalles menores ni secundarios.\n\n"
        "Texto:\n{text}\n\n"
        "Datos extraídos y resumen ejecutivo breve:"
    )
)

def extract_data_summary(text, extraction_template):
    """
    Extrae datos específicos y genera un resumen ejecutivo breve.
    """
    chain = LLMChain(llm=llm, prompt=extraction_prompt)
    result = chain.run(text=text, template_info=extraction_template)
    return result


# 3. Clasificación de la resolución

In [7]:
CATEGORIAS_CTMSG = {
    "Reglamentación y Normativas": [
        "REGLAMENTO",
        "MANUAL DE NORMA",
        "MANUAL Normas Compras y Contrataciones",
        "NORMA ISO",
        "LICENCIA ESPECIAL",
        "ESTATUTO DEL PERSONAL",
        "SUMARIO ADMINISTRATIVO"
    ],
    "Administración Financiera y Contable": [
        "REMUNERACIONES",
        "REGISTRACION CONTABLE",
        "BALANCES",
        "BANCOS",
        "PRESUPUESTOS",
        "TARIFAS ENERGÍA"
    ],
    "Contrataciones y Convenios": [
        "CONVENIO",
        "CONTRATACIONES Y COMPRAS",
        "COMPRAS AUTOMOTORES",
        "CONCURSOS PRECIOS",
        "PODERES ABOGADOS"
    ],
    "Infraestructura y Mantenimiento": [
        "EDIFICIOS",
        "VIVIENDAS",
        "EQUIPAMIENTO REPRESA",
        "MANTENIMIENTO",
        "TRANSFORMADORES",
        "SEGURIDAD PRESA",
        "PUERTO SECO"
    ],
    "Servicios Médicos y Sociales": [
        "PRESTAMOS PERSONAL",
        "SERVICIOS MÉDICOS",
        "ATENCIÓN MÉDICA",
        "SEGUROS VIDA"
    ],
    "Comunicaciones y Relaciones Institucionales": [
        "RELACIONES PÚBLICAS",
        "COMUNICACIÓN INSTITUCIONAL",
        "PUBLICIDAD",
        "LOGOTIPO",
        "FIRMA DIGITAL",
        "PAGINA WEB"
    ],
    "Auditorías y Control Interno": [
        "AUDITORIA INTERNA",
        "AUDITORÍA EXTERNA",
        "CONTROL GESTIÓN"
    ],
    "Recursos Humanos": [
        "CONTROL ASISTENCIA",
        "EPIDEMIOLOGÍA",
        "CURSOS Y SEMINARIOS",
        "DESVINCULACIONES"
    ],
    "Ambiental y Social": [
        "RIBEREÑOS",
        "MERCOSUR",
        "MERCADOS ELÉCTRICOS",
        "CULTURALES",
        "SAOTTA"
    ],
    "Gestión y Operaciones": [
        "DIRECCIÓN EJECUTIVA",
        "GERENCIA",
        "DELEGACIONES",
        "TAISG",
        "HORARIOS OFICINAS",
        "GRUPOS TRABAJO",
        "CENTRO DOCUMENTACIÓN",
        "RENOVACIÓN SALTO GRANDE"
    ]
}


In [8]:
# Prompt adaptado
classification_prompt = PromptTemplate(
    input_variables=["text", "data_summary", "categorias"],
    template=(
        "Basándote en el siguiente texto y en los datos extraídos y resumen:\n\n"
        "Texto: {text}\n\n"
        "Datos y resumen: {data_summary}\n\n"
        "Considerando estas categorías disponibles: {categorias}\n\n"
        "Clasifica la resolución estrictamente dentro de una de estas categorías principales."
    )
)

# Función ajustada
def classify_resolution(text, data_summary):
    categorias_str = ', '.join(CATEGORIAS_CTMSG.keys())
    chain = LLMChain(llm=llm, prompt=classification_prompt)
    classification = chain.run(text=text, data_summary=data_summary, categorias=categorias_str)
    return classification


# Función principal para procesar el PDF y guardar la información en TinyDB

In [9]:
import re

def parse_data_summary(data_summary):
    """
    Divide el resumen extraído en campos separados.
    """
    fields = ["Número de resolución", "Fecha", "Ente emisor", "Resumen ejecutivo breve"]
    parsed_data = {}

    for field in fields:
        pattern = rf"{field}:\s*(.+)"
        match = re.search(pattern, data_summary)
        if match:
            parsed_data[field] = match.group(1).strip()
        else:
            parsed_data[field] = None

    return parsed_data

In [10]:
# %% [code]
def process_pdf(pdf_path, extraction_template, db_table):
    """
    Procesa un archivo PDF completo: extrae el texto, lo corrige,
    extrae datos y resumen, clasifica la resolución y guarda todo en TinyDB.
    """
    file_name = os.path.basename(pdf_path)
    
    # Verificar si el documento ya se encuentra en la base de datos
    Record = Query()
    if db_table.search(Record.pdf_file == file_name):
        print(f"El documento {file_name} ya se encuentra registrado en TinyDB.")
        return

    # Extracción de texto
    raw_text = pdf_to_text(pdf_path)
    print(f"Texto extraído del archivo {file_name}.")
    
    # Corrección de la transcripción
    corrected_text = correct_transcription(raw_text)
    print("Transcripción corregida.")
    
    # Extracción de datos y resumen
    extracted_result = extract_data_summary(corrected_text, extraction_template)
    print("Datos y resumen extraídos.")
    
    # Clasificación de la resolución
    classification = classify_resolution(corrected_text, extracted_result)
    print("Clasificación realizada.")
    
    # Crear el registro con la información resultante
    parsed_summary = parse_data_summary(extracted_result)

    record = {
        "pdf_file": file_name,
        "text": corrected_text,
        "numero_resolucion": parsed_summary["Número de resolución"],
        "fecha": parsed_summary["Fecha"],
        "ente_emisor": parsed_summary["Ente emisor"],
        "resumen_ejecutivo": parsed_summary["Resumen ejecutivo breve"],
        "classification": classification
    }
    
    # Guardar el registro en TinyDB
    db_table.insert(record)
    print(f"El documento {file_name} ha sido registrado en TinyDB.")

    return record


# Ejecución del procesamiento

In [11]:
# %% [code]
# Template explícito para extracción
extraction_template = (
    "- Número de resolución:\n"
    "- Fecha:\n"
    "- Ente emisor:\n"
    "- Resumen ejecutivo breve (máximo 30 palabras, tema principal solamente):"
)

# Carpeta donde se encuentran los archivos PDF
pdf_folder = "pdfs"  # Asegúrate de tener esta carpeta y colocar allí tus PDFs

# Procesar cada PDF nuevo en la carpeta
for file_name in os.listdir(pdf_folder):
    if file_name.lower().endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, file_name)
        process_pdf(pdf_path, extraction_template, records_table)


El documento res00222.pdf ya se encuentra registrado en TinyDB.
El documento res00122.pdf ya se encuentra registrado en TinyDB.


# Función para procesar un nuevo PDF individualmente

In [ ]:
# %% [code]
def procesar_nuevo_pdf(pdf_path, extraction_template, db_table):
    """
    Procesa un nuevo PDF individualmente y lo añade a la base de datos TinyDB.
    """
        # Crear el registro con la información resultante
    record = {
        "pdf_file": "New Resolucion",
        "text": "corrected_text",
        "data_summary": "extracted_result",
        "classification": "classification"
    }

    file_name = os.path.basename(pdf_path)
    Record = Query()
    if db_table.search(Record.pdf_file == file_name):
        print(f"El documento {file_name} ya se encuentra registrado.")
        return
    else:
        record = process_pdf(pdf_path, extraction_template, db_table)
        print(record)
        

#Ejemplo de llamada para procesar un nuevo PDF:
nuevo_pdf = "pdfs/new/res00422.pdf"
procesar_nuevo_pdf(nuevo_pdf, extraction_template, records_table)



Texto extraído del archivo res00422.pdf.


/var/folders/66/98ml0k7s50sbjln7zqxcj2y80000gn/T/ipykernel_10972/3386155856.py:11: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=correction_prompt)
/var/folders/66/98ml0k7s50sbjln7zqxcj2y80000gn/T/ipykernel_10972/3386155856.py:12: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  corrected = chain.run(text=text)


# Inspeccion de DB

In [15]:
import pandas as pd

db = TinyDB("ResCTM.json")
records_table = db.table("records")

registro_4 = records_table.get(doc_id=2)
if registro_4:
    for campo, valor in registro_4.items():
        print(f"{campo}:")
        if isinstance(valor, str) and len(valor) > 150:
            print(valor[:300] + "...")
        else:
            print(valor)
        print("-" * 50)
else:
    print("Registro no encontrado.")

print(registro_4)

pdf_file:
res00222.pdf
--------------------------------------------------
text:
COMISIÓN TÉCNICA MIXTA DE SALTO GRANDE  
Videoconferencia, 26 de enero de 2022  
RESOLUCIÓN CTM Nº 002/22 (Acta Nº 1127 - as. 8.2) 
 
VISTO: el EE 2021-SCYC-0144 en el que tramita la Resolución CTM Nº 173/21 de fecha 15.09.21, que deja sin efecto la Resolución CTM Nº 140/21 y aprueba el Pliego de Ba...
--------------------------------------------------
numero_resolucion:
CTM Nº 002/22
--------------------------------------------------
fecha:
26 de enero de 2022
--------------------------------------------------
ente_emisor:
Comisión Técnica Mixta de Salto Grande
--------------------------------------------------
resumen_ejecutivo:
Se declara fracasada la Licitación Pública del Contrato SG-723 “Reconversión de monte forestal en ambientes nativos”.
--------------------------------------------------
classification:
La resolución CTM Nº 002/22, que declara fracasada la Licitación Pública del Contrato SG-723 “Re

In [13]:
#db = TinyDB('historial.json')
#db.truncate()  # Esto vacía completamente la base de datos
#db.close()